This is a project following the video tutorial named "Algorhitmic Trading Using Python - Full Course"

You can access it here: https://www.youtube.com/watch?v=xfzGZB4HhEE&t=3272s

# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [20]:
import numpy as np
import yfinance as yf
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [21]:
stocks = pd.read_csv("SP 500 Stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [22]:
stocks = stocks["Ticker"].tolist()

In [23]:
data = yf.download(stocks, start="2024-09-22", end="2024-09-22")

[*********************100%***********************]  505 of 505 completed

505 Failed downloads:
['UAA', 'AMZN', 'BAC', 'SLG', 'MET', 'MCK', 'CAT', 'FANG', 'SRE', 'PNR', 'WHR', 'IFF', 'EXPE', 'MDLZ', 'KLAC', 'FMC', 'BK', 'VNO', 'FITB', 'ICE', 'BKNG', 'AME', 'AMT', 'GRMN', 'CF', 'HUM', 'JKHY', 'DG', 'MAR', 'STX', 'ED', 'AEE', 'WFC', 'CMS', 'ADI', 'MTB', 'NTRS', 'ILMN', 'CPRT', 'STZ', 'GILD', 'MCO', 'CINF', 'TFX', 'SWK', 'SIVB', 'TPR', 'TDG', 'ALK', 'FAST', 'EXR', 'CNP', 'CAG', 'PPL', 'ROK', 'CTSH', 'ZBH', 'WAB', 'PEG', 'OMC', 'DTE', 'AMCR', 'NVDA', 'REG', 'TSN', 'JCI', 'ATO', 'RL', 'SNPS', 'ESS', 'ROL', 'GWW', 'NOC', 'XEL', 'KHC', 'FTV', 'J', 'PKG', 'LH', 'DD', 'VZ', 'CFG', 'KEYS', 'USB', 'WELL', 'EMR', 'RHI', 'KMX', 'GD', 'GPS', 'IP', 'EXPD', 'INCY', 'XYL', 'SJM', 'LUV', 'CMCSA', 'HIG', 'MMM', 'AVY', 'LYB', 'MCHP', 'ROP', 'AJG', 'BMY', 'DHI', 'CRM', 'RF', 'MSI', 'EA', 'DIS', 'PSX', 'TMO', 'TTWO', 'FTNT', 'GOOGL', 'DVA', 'HII', 'BBY', 'EOG', 'CMA', 'NWS', 'HBI', 'HCA', 'NOV', 'GPC', 'WM'

Since we have some data that failed to download, we should drop it out of our analysis.

In [24]:
data = data.dropna(axis=1)
price = data["Close"]
#price = price.iloc[0]
price

Ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,


We're left with 459 stocks, which will be good enough for this project.

In [25]:
symbol = data["Close"].T
symbol = symbol.index
symbol

Index(['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE',
       ...
       'XLNX', 'XOM', 'XRAY', 'XRX', 'XYL', 'YUM', 'ZBH', 'ZBRA', 'ZION',
       'ZTS'],
      dtype='object', name='Ticker', length=505)

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price to a pandas DataFrame. 

In [26]:
my_columns = ["Number Of Shares To Buy"]
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe.insert(loc=0, column="Ticker", value=symbol)
final_dataframe.insert(loc=1, column="Price", value=price)
final_dataframe

ValueError: Expected a one-dimensional object, got a DataFrame with 505 columns instead.

Note: this could've been done by just extracting "Ticker" and "Close" from our data frame and adding a column named "Number of Shares To Buy". This however, is an exercise to get more comfortable with data wrangling.

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [ ]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("Not a number!")
    input("Enter the value of your portfolio:")
    val = float(portfolio_size)

In [ ]:
portfolio_size

'1000'

In [ ]:
position_size = val/len(final_dataframe.index)
print(position_size)

NameError: name 'final_dataframe' is not defined

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: